# Support Vector Machine
SVM is a machine learning algorithm used for classification and partially for regression.

* **Basic idea**: Find the boundary (hyperplane) that best separates the different classes in the data.
* **Maximizing the edge**: SVM tries to find the hyperplane so that the distance between the boundary and the closest points of the two classes is as large as possible (these points are called support vectors).
* **Linearly separable data**: If the classes can be separated by a line (2D) or a hyperplane (multiple dimensions), SVM finds the optimal separating hyperplane.
* **Linear data**: Using a kernel trick (e.g. RBF, polynomial), data can be mapped to a higher dimension where it is linearly separable.
* **Regularization**: The C parameter affects the trade-off between maximizing the edge and minimizing classification errors on the training data.

* Advantages:
    * Works well for high-dimensional data.
    * Robust to overfitting, especially if the kernel and C parameter are set correctly.

* Disadvantages:
    * Can be slow on large datasets.
    * Choosing the right kernel and parameters requires experimentation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Reading prepared data from numpy file

In [ ]:
my_arrays = np.load("iris_numpy.npz")
X = my_arrays['arr_0']
Y = my_arrays['arr_1']
X_features = my_arrays['arr_2']

Splitting of data into training and test data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split (X, Y, test_size=0.2)

Loading encoder and scaler

In [ ]:

import joblib
scaler=joblib.load('classification_std_scaler.bin')
encoder=joblib.load('classification_encoder.bin')

## Model training
Creating and training a linear SVM model

In [ ]:
from sklearn import svm

svm_model = svm.SVC(kernel = 'linear', random_state = 0)
svm_model.fit(X_train, Y_train)

Running the model on test data

In [ ]:
Y_pred = svm_model.predict(X_test)

## Model validation

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = svm_model, X = X_train, y = Y_train)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
cf_matrix = confusion_matrix(Y_test, Y_pred)
sns.heatmap(cf_matrix, annot=True)

Accuracy

In [ ]:
accuracy_score(Y_test, Y_pred)

## Model visualization

In [ ]:
from sklearn.inspection import DecisionBoundaryDisplay

Support vectors are key to SVM. These are the points that define the boundary.

In [ ]:
svm_model.support_vectors_

We will prepare a function that displays the measured values, support vectors and class membership using color as well as class boundaries.

In [ ]:
def SVM_vizualization (svm_model, X, Y, title, xlabel, ylabel):
    # Display of decision boundaries
    disp = DecisionBoundaryDisplay.from_estimator(
        svm_model,
        X,
        response_method="predict",
        cmap=plt.cm.coolwarm,
        alpha=0.8,        
    )
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)    

    # View points
    plt.scatter(X[:, 0], X[:, 1], c=Y, cmap=plt.cm.coolwarm, s=20, edgecolors="k")

    # View support vectors - larger circles
    try:
        plt.scatter(svm_model.support_vectors_[:,0], svm_model.support_vectors_[:,1] , s=40, edgecolors="k", facecolors='none')
    except:
        pass

In [ ]:
SVM_vizualization (svm_model, X_train, Y_train, "Linear kernel", "Petal length", "Petal width")

## Other kernels
SVM control parameter C is the margin sensitivity to point distance, defines the penalty if a point is misaligned.

In [ ]:
C = 1.0

### Linear kernel
Let's try calling SVM with a different kernel. Let's try linear.

In [ ]:
svm_linearsvc=svm.LinearSVC(C=C, max_iter=10000)
svm_linearsvc.fit(X_train, Y_train)
SVM_vizualization (svm_linearsvc, X_train, Y_train, "LinearSVC (linear kernel)", "Petal length", "Petal width")
print (f"Score: {svm_linearsvc.score(X_test, Y_test)}")

### RBF kernel

In [ ]:
C = 1.0
svc_rbf=svm.SVC(kernel="rbf", gamma=0.7, C=C)
svc_rbf.fit(X_train, Y_train)
SVM_vizualization (svc_rbf, X_train, Y_train, "RBF kernel", "Petal length", "Petal width")
print (f"Score: {svc_rbf.score(X_test, Y_test)}")

### Polynomial kernel
The degree of the polynomial is set to 3.

In [ ]:
svc_poly=svm.SVC(kernel="poly", degree=3, gamma="auto", C=C)
svc_poly.fit(X_train, Y_train)
SVM_vizualization (svc_poly, X_train, Y_train, "Poly kernel", "Petal length", "Petal width")
print (f"Score: {svc_poly.score(X_test, Y_test)}")

## Hyperparameters
Similar to k-means, SVM has multiple parameters that affect the accuracy of the model. Correct settings can be crucial, but sometimes difficult to estimate.

One approach is to try to create SVM models for all combinations of parameters and observe which has the best result.

There may be a large number of models to create, so this can take a while.

The scikit library already has a function for this.

In [ ]:
from sklearn.model_selection import GridSearchCV
svc=svm.SVC()
params=[
    {"kernel": ["poly"], "C":[1, 5, 10, 50], "degree":[1,2,3,4], "gamma":[0.1, 0.3, 0.7, 1, 5]},
    {"kernel": ["rbf"], "C":[1, 5, 10, ], "gamma":[0.1, 0.5, 1, 5]},    
]

In [ ]:
clf=GridSearchCV(estimator=svc, param_grid=params, cv=5, verbose=1)
clf.fit(X_train, Y_train)

Display the score of the best model.

In [ ]:
clf.score(X_test, Y_test)

Display the input parameters of the best model.

In [ ]:
clf.best_params_

And its visualization

In [ ]:
SVM_vizualization (clf, X_train, Y_train, "best SVM", "Petal length", "Petal width")